In [1]:
import requests
from config import CLIENT_SECRET, CLIENT_ID
import base64
from requests import post , get
import json
from IPython.display import display
import pandas as pd 

In [2]:
def get_token():
     # Spotify API isteklerini yetkilendirmek için kullanılan istemci kimlik bilgileri
    auth_string = CLIENT_ID + ":" + CLIENT_SECRET
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"

    headers = {
         "Authorization": "Basic " + auth_base64,
         "Content-Type" : "application/x-www-form-urlencoded"
     }
    # Token almak için kullanılan istek verisi
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content.decode("utf-8"))
    token = json_result["access_token"]
    return token


In [3]:
def get_auth_header(token):
    """
    Erişim token'ını içeren yetkilendirme başlığını oluşturur.

    Args:
    - token (str): Spotify API'ye erişim için kullanılan erişim token'ı.

    Returns:
    - dict: API isteklerinde kullanılacak yetkilendirme başlığı.
    """
    return {'Authorization': 'Bearer ' + token}

In [4]:
# Erişim token'ını almak için get_token fonksiyonunu kullan
token = get_token()

In [5]:
# Kullanıcıya ait çalma listelerini getiren fonksiyon
def get_user_playlists(token, user_id, limit=50):
    url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
    headers = get_auth_header(token)
    params = {'limit': limit}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        playlists_data = response.json()
        return playlists_data.get('items', [])
    else:
        print(f"Hata Kodu: {response.status_code}, Hata Mesajı: {response.text}")
        return None

user_id = 'arzumerturkk'

playlists_data = get_user_playlists(token, user_id)

# Tüm çalma listelerini depolamak için boş bir sözlük oluştur
all_playlists = {'playlist_owner':[], 'playlist_name':[], 'playlist_id':[], 'number_of_songs':[]}

# Eğer çalma listesi verisi mevcutsa
if playlists_data:
    # Her çalma listesi için gerekli bilgileri çıkart
    for playlist in playlists_data:
        all_playlists['playlist_owner'].append(playlist['owner']['display_name'])
        all_playlists['playlist_name'].append(playlist['name'])
        all_playlists['playlist_id'].append(playlist['id'])
        all_playlists['number_of_songs'].append(playlist['tracks']['total'])

# Elde edilen bilgileri DataFrame'e dönüştür
all_playlists_df = pd.DataFrame(all_playlists)

all_playlists_df

,playlist_owner,playlist_name,playlist_id,number_of_songs
0,arzumerturkk,fav,6xQVbLDvgluW1PzPddWYNv,49
1,arzumerturkk,2023 Şişede Saklı Listem,7K95g0wPc0Bgv8Z1qRDFj0,3
2,arzumerturkk,chill,0YyiSXuuUzH3edZDrIHKzB,41
3,Umut K.,STY,3faPrXWCGVbNKGnJTfTNnG,43
4,elif taştekne,karışık kaset,5KoWCjodAbEVxZrrQ1S7qd,234


In [6]:
# Belirtilen çalma listesindeki şarkıları getiren fonksiyon.
def get_playlist_items(token, playlist_id, market='US', limit=50):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = get_auth_header(token)
    params = {'market': market, 'limit': limit}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        tracks_data = response.json()
        return tracks_data.get('items', [])
    else:
        print(f"Hata Kodu: {response.status_code}, Hata Mesajı: {response.text}")
        return None


In [7]:
# Tüm çalma listelerinin ID'lerini ekrana yazdır
for playlist_id in all_playlists_df['playlist_id']:
    print(playlist_id)

6xQVbLDvgluW1PzPddWYNv
7K95g0wPc0Bgv8Z1qRDFj0
0YyiSXuuUzH3edZDrIHKzB
3faPrXWCGVbNKGnJTfTNnG
5KoWCjodAbEVxZrrQ1S7qd


In [8]:

all_playlists_items = {'name':[], 'artist':[], 'duration_ms':[], 'popularity':[], 'added_date':[], 'id':[]}

for playlist_id in all_playlists_df['playlist_id']:
    playlist_tracks = get_playlist_items(token, playlist_id)
    
    if playlist_tracks:
        for track in playlist_tracks:
            all_playlists_items['name'].append(track['track']['name'])
            all_playlists_items['artist'].append(track['track']['artists'][0]['name'])
            all_playlists_items['duration_ms'].append(track['track']['duration_ms'])
            all_playlists_items['popularity'].append(track['track']['popularity'])
            all_playlists_items['added_date'].append(track['added_at'])
            all_playlists_items['id'].append(track['track']['id'])
            
all_playlists_items_df = pd.DataFrame(all_playlists_items)

all_playlists_items_df.head()
         

,name,artist,duration_ms,popularity,added_date,id
0,Anemone,The Brian Jonestown Massacre,334466,59,2023-12-26T08:38:00Z,70WSr32oW4jC1na5I0j0D6
1,Ağlama ben ağlarım,Canozan,191694,80,2023-12-26T08:38:06Z,0JU2QfhTrgFsVq5yQ4YQlu
2,Kırık Ayna,Cihat Akbel,128391,32,2023-12-26T08:38:11Z,4FWTxZMrfs4EykvjhkIRjV
3,Tek Başına,Erkin Koray,297200,19,2023-12-26T08:38:16Z,0CrzShyasw5dgt2mZoYAxW
4,Beklediğim Gibiyim,Büyük Ev Ablukada,236631,56,2023-12-26T08:38:20Z,4U0mwukUqiKG9ucB4f2Ex8


In [9]:
# Verilen şarkı ID'leri için Spotify API'sinden ses özelliklerini alır
def get_audio_features(song_ids, token):
    url = 'https://api.spotify.com/v1/audio-features'
    headers = get_auth_header(token)
    params = {'ids': song_ids}

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        audio_features = response.json()
        return audio_features['audio_features']
    else:
        print(f'Hata kodu: {response.status_code}')
        print(response.text)
        return None

In [10]:

all_features = {'Danceability':[], 'Energy':[], 'Instrumentalness':[], 'Loudness':[], 'speechiness':[], 'tempo':[], 'valence':[], 'acousticness':[], 'mode':[], 'id':[]}

# Her bir şarkı için audio özelliklerini al
for song_ids in all_playlists_items_df['id']:
    result = get_audio_features(song_ids, token)

    if result:
        for features in result:
            all_features['Danceability'].append(features['danceability'])
            all_features['Energy'].append(features['energy'])
            all_features['Instrumentalness'].append(features['instrumentalness'])
            all_features['Loudness'].append(features['loudness'])
            all_features['speechiness'].append(features['speechiness'])
            all_features['tempo'].append(features['tempo'])
            all_features['valence'].append(features['valence'])
            all_features['acousticness'].append(features['acousticness'])
            all_features['mode'].append(features['mode'])
            all_features['id'].append(features['id'])
            
# Tüm özellikleri içeren DataFrame oluştur
all_features_df = pd.DataFrame(all_features)
all_features_df.head()
           

,Danceability,Energy,Instrumentalness,Loudness,speechiness,tempo,valence,acousticness,mode,id
0,0.527,0.636,0.843000,-12.888,0.0347,83.101,0.607,0.0599,0,70WSr32oW4jC1na5I0j0D6
1,0.416,0.473,0.000001,-9.726,0.0351,118.354,0.167,0.5530,0,0JU2QfhTrgFsVq5yQ4YQlu
2,0.515,0.678,0.005910,-7.894,0.1740,124.847,0.880,0.5840,0,4FWTxZMrfs4EykvjhkIRjV
3,0.395,0.759,0.000000,-9.705,0.1270,171.224,0.587,0.2370,0,0CrzShyasw5dgt2mZoYAxW
4,0.746,0.562,0.747000,-11.574,0.0676,106.076,0.642,0.0737,0,4U0mwukUqiKG9ucB4f2Ex8


In [11]:
# Id' leri baz alarak şarkı özellikleri ile şarkı bilgilerini birleştir
df = pd.merge(all_playlists_items_df, all_features_df, on='id', how='inner')
df.shape

(204, 15)

In [14]:
# Birleştirilmiş DataFrame'i CSV dosyasına kaydet
df.to_csv('spotify_data.csv', index=False)